In [1]:
import numpy as np
import pandas as pd
import pickle
import warnings 
warnings.filterwarnings("ignore")
from flask import Flask, request, jsonify, render_template

In [2]:
#LabelEncoders
shipModeenc = pickle.load(open("Ship Modeencoder.pkl","rb"))
segenc = pickle.load(open("Segmentencoder.pkl","rb"))
cenc = pickle.load(open("Cityencoder.pkl","rb"))
senc = pickle.load(open("Stateencoder.pkl","rb"))
renc = pickle.load(open("Regionencoder.pkl","rb"))
catenc = pickle.load(open("Categoryencoder.pkl","rb"))
scatenc = pickle.load(open("Sub-Categoryencoder.pkl","rb"))

In [3]:
#pipeline
spipe = pickle.load(open("Salespipeline.pkl","rb"))
qpipe = pickle.load(open("Quantitypipeline.pkl","rb"))
dpipe = pickle.load(open("Discountpipeline.pkl","rb"))

In [4]:
#model
model = pickle.load(open("final_model.pkl","rb")) #Unpickling our created model
model

GradientBoostingRegressor(max_depth=None, max_features=6, min_samples_leaf=19,
                          n_estimators=90)

In [5]:
def ValuePredictor(to_predict_list):
    to_predict_list = list(to_predict_list.values())
    to_predict_list[0] = shipModeenc.transform([to_predict_list[0]])
    to_predict_list[1] = segenc.transform([to_predict_list[1]])
    to_predict_list[2] = cenc.transform([to_predict_list[2]])
    to_predict_list[3] = senc.transform([to_predict_list[3]])
    to_predict_list[5] = renc.transform([to_predict_list[5]])
    to_predict_list[6] = catenc.transform([to_predict_list[6]])
    to_predict_list[7] = scatenc.transform([to_predict_list[7]])
    to_predict_list[8] = spipe.transform(np.array([to_predict_list[8]]).reshape(-1,1))
    to_predict_list[9] = qpipe.transform(np.array([to_predict_list[9]]).reshape(-1,1))
    to_predict_list[10] = dpipe.transform(np.array([to_predict_list[10]]).reshape(-1,1))
    to_predict = np.array([to_predict_list])
    result = model.predict(to_predict)
    return result[0]

In [6]:
app = Flask(__name__) #Initializing Flask app
@app.route('/')
def DefaultFunction(): #default function gets called whenever we open this url
    return render_template("index.html")

@app.route("/predict",methods=["POST","GET"])
def predict():
    inp_features = request.form.to_dict()
    output = ValuePredictor(inp_features)
    if output>0:
        return render_template("index.html",pred=f"Predicted profit : {round(output,3)}")
    elif output==0:
        return render_template("index.html",pred="Neither profit nor loss")
    elif output<0:
        return render_template("index.html",pred=f"Predicted loss : {abs(round(output,3))}")
if __name__=="__main__":
    app.run(debug=False)
#Devices should be connected on the same wifi
#PC acts as server and all other devices can act clients on specifying the host address (your laptop IP Address)

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
